In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [2]:
aggSession = SparkSession.builder.appName("AggregationExample").getOrCreate()
df_agg=aggSession.read.option("header","true").option("inferSchema","true").csv("train.csv")
df_agg.show(5)

25/11/29 18:52:42 WARN Utils: Your hostname, Shithils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.226 instead (on interface en0)
25/11/29 18:52:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/29 18:52:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|    1|        5|2015-07-31| 5263|      555|   1|    1|           0|            1|
|    2|        5|2015-07-31| 6064|      625|   1|    1|           0|            1|
|    3|        5|2015-07-31| 8314|      821|   1|    1|           0|            1|
|    4|        5|2015-07-31|13995|     1498|   1|    1|           0|            1|
|    5|        5|2015-07-31| 4822|      559|   1|    1|           0|            1|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+
only showing top 5 rows



Spark is distributed, operations like distinct, join, groupby and repartition cause a shuffle which are expensive.
Shuffling happens when data needs to be moved between executors (nodes) in a distributed cluster to be reorganized.

In [3]:
df_agg.groupBy("Store").agg(sum("Sales").alias("TotalSales"), avg("Sales").alias("AvgSales"), count("Sales").alias("CountSales")).show(10)

+-----+----------+------------------+----------+
|Store|TotalSales|          AvgSales|CountSales|
+-----+----------+------------------+----------+
|  148|   6772949| 7189.967091295117|       942|
|  463|   4747749| 5040.073248407643|       942|
|  471|   4422266|  5834.12401055409|       758|
|  496|   6059458| 6432.545647558386|       942|
|  833|   4293342|4557.6878980891715|       942|
| 1088|   3960984| 4204.866242038216|       942|
|  243|   4407146| 5814.176781002639|       758|
|  392|   5670675| 6019.824840764331|       942|
|  540|   3513672| 4635.451187335092|       758|
|  623|   5473895| 5810.928874734607|       942|
+-----+----------+------------------+----------+
only showing top 10 rows



**Multi-Column GroupBy**

In [7]:
df_agg.groupBy("Store","DayofWeek").agg(sum("Sales").alias("TotalSales"), avg("Sales").alias("AvgSales"), count("Sales").alias("CountSales")).orderBy("Store","DayofWeek").show(10)

+-----+---------+----------+------------------+----------+
|Store|DayofWeek|TotalSales|          AvgSales|CountSales|
+-----+---------+----------+------------------+----------+
|    1|        1|    662780| 4946.119402985075|       134|
|    1|        2|    627874| 4650.918518518519|       135|
|    1|        3|    601354| 4454.474074074074|       135|
|    1|        4|    552772| 4094.607407407407|       135|
|    1|        5|    609716| 4516.414814814815|       135|
|    1|        6|    662358|4942.9701492537315|       134|
|    1|        7|         0|               0.0|       134|
|    2|        1|    775930| 5790.522388059701|       134|
|    2|        2|    719473|  5329.42962962963|       135|
|    2|        3|    770213| 5705.281481481482|       135|
+-----+---------+----------+------------------+----------+
only showing top 10 rows

